# (Semi-)Auto-calibration and threshold setting

This is a brief notebook for checking previous calibrations, 
doing new calibrations (on all or a subset of SiPMs), 
setting thresholds and putting all this into an override file.

See autocalib.ipynb for an extensive example of doing different checks

In [ ]:
%matplotlib inline

import glob
import os
from collections.abc import Callable, Sequence, Iterator, Mapping
from abc import ABC, abstractmethod
from typing import Any

import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.axes import Axes

import numpy as np
import awkward as ak

from lgdo import lh5
from lgdo.lh5.exceptions import LH5DecodeError
from legendmeta import LegendMetadata
from dspeed.processors import get_multi_local_extrema

from sipm_autocalib import *

plt.rcParams["figure.figsize"] = (10, 4)

In [ ]:
# metadata dir has to be declared save git directory

proj_dir = "/mnt/atlas02/projects/legend/sipm_qc"
dsp_dir = os.path.join(proj_dir, "data/tier/dsp/ssc/p16/r008")
dsp_files = glob.glob(dsp_dir+"/l200-*-tier_dsp.lh5")
dsp_files.sort()
if len(dsp_files) == 0:
    raise RuntimeError(f"Found no files in {dsp_dir}")
#dsp_files = dsp_files[:50]
metadata_dir = os.path.join(proj_dir, "metadata/legend-metadata-schwarz")
lmeta  = LegendMetadata(metadata_dir)
chmap = lmeta.channelmap(get_timestamp_from_filename(dsp_files[0]))
chmap_sipm = chmap.map("system", unique=False).spms
#requires recent legend-datasets
raw_keys = chmap_sipm.map("analysis.usability", unique=False).on.map("daq.rawid").keys()
onlythese = ["S011", "S080", "S085", "S086", "S092"]
if len(onlythese) > 0:
    raw_keys = [chmap[k].daq.rawid for k in onlythese]

In [ ]:
energies = get_energies(dsp_files, raw_keys, chmap, take_pulser_from_normal=True)

In [ ]:
# check here if we need a recalibration:
if False:
    calib, thresholds = read_override_file(os.path.join(metadata_dir, "dataprod/overrides/hit/lar/p15/r004/l200-p15-r004-lar-T%-par_hit-overwrite.yaml"))
    calib_p16, ths_p16 = read_override_file(os.path.join(metadata_dir, "dataprod/overrides/hit/lar/p16/r000/l200-p16-r000-lar-T%-par_hit-overwrite.yaml"))
    calib = calib | calib_p16 # merge re-calibrated SiPMs in
    thresholds = thresholds | ths_p16 # also thresholds
    calibrated_histos = get_calibrated_histograms(energies, calib, (0,5), 200)
    _ = plot_all_pe_histograms_and_thresholds(calibrated_histos, thresholds, gridx=True)

if False:
    calib, thresholds = read_override_file("../out/temp_override.yaml")
    calibrated_histos = get_calibrated_histograms(energies, calib, (0,5), 200)
    _ = plot_all_pe_histograms_and_thresholds(calibrated_histos, thresholds, gridx=True)

In [ ]:
config = load_config_file(os.path.join(proj_dir, "sipm-autocalib/config/autocalib-p16-r008-conf.yaml"))

gen_hist_defaults = config["gen_hist_defaults"]
peakfinder_defaults = config["peakfinder_defaults"]
simple_calibration_defaults = config.get("simple_calibration_defaults", {})
advanced_calibration_defaults = config["advanced_calibration_defaults"]
gen_hist_overrides = config.get("gen_hist_overrides", {})
peakfinder_overrides = config.get("peakfinder_overrides", {})
simple_calibration_overrides = config.get("simple_calibration_overrides", {})
advanced_calibration_overrides = config.get("advanced_calibration_overrides", {})

In [ ]:
gen_hist_defaults

In [ ]:
simple_calib_output, _, _ = multi_simple_calibration(energies, gen_hist_defaults, peakfinder_defaults, {}, peakfinder_overrides=peakfinder_overrides, draw=True, nodraw_axes=True)

In [ ]:
calibrated_histos = get_calibrated_histograms(energies, simple_calib_output, (0, 5), 200)
_ = plot_all_pe_histograms(calibrated_histos)

In [ ]:
advanced_calib_output, _, _ = multi_advanced_calibration(calibrated_histos, get_calibrated_PE_positions(simple_calib_output), advanced_calibration_defaults, calibration_overrides=advanced_calibration_overrides, draw=True)

In [ ]:
adv_calibrated_histos = get_calibrated_histograms(energies, combine_multiple_calibrations(simple_calib_output, advanced_calib_output), (0, 5), 200)
thresholds = multi_valley_minima(adv_calibrated_histos, mimimum_position=0.4)
_ = plot_all_pe_histograms_and_thresholds(adv_calibrated_histos, thresholds, gridx=True)

## Store the final data into an override file

In [ ]:
if False:
    output_override_file("out/temp_override.yaml", combine_multiple_calibrations(simple_calib_output, advanced_calib_output), thresholds)